Binary Decidion Tree

First lets import all libraries that we will later use.

In [3]:

import csv as csv

from random import seed
from random import randrange
%matplotlib inline

First what we do is create a constructor for our class. 
parameters:
    max_tree_lvl - This is the maximum depth of our tree.
    min_tree_size - This is the minimum number of training patterns that a given node is responsible for.
    seed - is the root node of our tree

In [5]:
class Decidion_Tree():

    def __init__(self, max_tree_lvl, min_tree_size):
        self.max_tree_lvl = max_tree_lvl
        self.min_tree_size = min_tree_size
        seed(1)

 index - index of sample element that we will compare to given value
 value - value which determines whether we add current sample elements to one or other class 
 data - all our given data set 
 This method determines how to group our given data samples depending on each samples certain element value

In [6]:
class Decidion_Tree(Decidion_Tree):    
    def split_into_groups(self, index, value, data):
        first, second = list(), list()
        for row in data:
            if row[index] > value:
                second.append(row)
            else:
                first.append(row)
        return first, second

 groups - our data samples that have been split into two groups based on a value
 classes - how many unique class labels do we have 

 First thing what we do is determine how any samples together are in both groups
 We initialize gini_index - the name of the cost function used to evaluate splits in the data set
 Now we count how many data samples are in each group 
 If group does not have any samples we continue
 But if we have data in our group what we do next is calculate: 
    Gini score that  gives an idea of how good a split is by how mixed the classes are in the two groups created by the split.
    for this score first we count how many certain class elements are in group than divide it with all the group sample size
    the result is multiplied with itself and added to the score
    Next step is gini index calculation where we substract from 1 our score and multiply it with groups sample size devided by all samples size
At the end we return gini index

In [7]:
class Decidion_Tree(Decidion_Tree):      
    def calculate_gini_index(self, groups, classes):
        sample_size = float(sum([len(group) for group in groups]))
        gini_index = 0.0
        for group in groups:
            group_sample_size = float(len(group))
            score = 0.0
            if group_sample_size != 0:
                for class_sample in classes:
                    p = [row[-1] for row in group].count(class_sample) / group_sample_size
                    score += p * p
                gini_index += (1.0 - score) * (group_sample_size / sample_size)
            else:
                continue
        return gini_index

 Data - our data set
 What this method does is finding the best split for our data set
 We determine parameters to use to best split our data by into two groups:
     what is the best index 
     what are the best groups 
     what is the best gini index

In [8]:
class Decidion_Tree(Decidion_Tree):      
    def split_into_classes(self, data):
        class_values = list(set(row[-1] for row in data))
        best_index = 100
        best_value = 100
        best_score = 100
        best_groups = None
        gini_index = 0
        for index in range(len(data[0]) - 1):
            for row in data:
                groups = self.split_into_groups(index, row[index], data)
                gini_index = self.calculate_gini_index(groups, class_values)
                if gini_index < best_score:
                    best_index, best_value, best_score, best_groups = index, row[index], gini_index, groups
        return {'index': best_index, 'value': best_value, 'groups': best_groups, 'gini': gini_index}

 terminal node method is used to determine when to stop growing our tree
 In other words when we stop growing at a given point, this node is called a terminal node and is used to make a final prediction.
 In our case 0 or 1
 Terminal node is calculate by counting how many times each class label in group repeats 
 As terminal node value is take the class label with biggest occurrence 

In [9]:
class Decidion_Tree(Decidion_Tree):      
    def set_terminal_value(self, group):
        result = [row[-1] for row in group]
        return max(set(result), key=result.count)

 Now what we do is a recursive tree building 
 First, the two groups of data split by the node are divided into left and right and deleted from the node. 
 As we work on these groups the node no longer requires access to these data.
 Next, we check if either left or right group of rows is empty 
 and if so we create a terminal node using what records we do have.
 We then check if we have reached our maximum depth 
 and if so we create a terminal node.
 We then process the left child, 
 creating a terminal node if the group of rows is too small, 
 otherwise create and add the left node in a depth first
  until the bottom of the tree is reached on this branch.
 The right side is then processed in the same manner, 
 as we rise back up the constructed tree to the root.

In [10]:
class Decidion_Tree(Decidion_Tree):      
    def create_tree_structure(self, node, level):
        left, right = node['groups']
        del (node['groups'])
        if not left or not right:
            node['left'] = node['right'] = self.set_terminal_value(left + right)
            return
        if level >= self.max_tree_lvl:
            node['left'], node['right'] = self.set_terminal_value(left), self.set_terminal_value(right)
            return
        if len(left) <= self.min_tree_size:
            node['left'] = self.set_terminal_value(left)
        else:
            node['left'] = self.split_into_classes(left)
            self.create_tree_structure(node['left'], level + 1)
        if len(right) <= self.min_tree_size:
            node['right'] = self.set_terminal_value(right)
        else:
            node['right'] = self.split_into_classes(right)
            self.create_tree_structure(node['right'], level + 1)

 Now what we do is build a binary decision tree by giving it training data needed to create the tree structure
 First we divide the training data into classes or in other words find best data split 
 Now we use this data to build our tree using the recursive tree building

In [11]:
class Decidion_Tree(Decidion_Tree):      
    def build_decision_tree(self, train):
        root_data = self.split_into_classes(train)
        self.create_tree_structure(root_data, 1)
        return root_data

 Testing sample class prediction also happens recursively 
 We decide in which branch to search for our samples call
 What we do is take a certain sample elements value 
 Based on the index that best describes the class this we calculated when we tried to find the best Gini index
 Than we do this till we find terminal node which is also our class label

In [12]:
class Decidion_Tree(Decidion_Tree):      
    def predict_class(self, node, row):
        if row[node['index']] < node['value']:
            if isinstance(node['left'], dict):
                return self.predict_class(node['left'], row)
            else:
                return node['left']
        else:
            if isinstance(node['right'], dict):
                return self.predict_class(node['right'], row)
            else:
                return node['right']

 This method is used for classifying our test sample data
 First we build our tree 
 Than we pass each sample and the tree (created binary decision tree from training data) to predict_class method 
 We return the predicted class and our tree structure

In [13]:
class Decidion_Tree(Decidion_Tree):      
    def decision_tree_classifier(self, train, test):
        tree = self.build_decision_tree(train)
        predictions = list()
        for row in test:
            prediction = self.predict_class(tree, row)
            predictions.append(prediction)
        return predictions, tree

 This is k-fold cross validation to evaluate the performance of the algorithm on the data set.
 The idea is to divide all our data into small chunks 

In [14]:
class Decidion_Tree(Decidion_Tree):      
    def split_into_validation_data(self, data, chunks):
        data_split = list()
        data_copy = list(data)
        chunk_size = int(len(data) / chunks)
        for i in range(chunks):
            chunk = list()
            while len(chunk) < chunk_size:
                index = randrange(len(data_copy))
                chunk.append(data_copy.pop(index))
            data_split.append(chunk)
        return data_split

 This method is used to measure how good our binary decision tree is working 
 we count all the correctly found labels 
 Then divide them with actual label number and multiply with 100 to get percentage

In [15]:
class Decidion_Tree(Decidion_Tree):      
    def prediction_accuracy(self, actual_labels, predicted_labels):
        counter = 0
        for i in range(len(actual_labels)):
            if actual_labels[i] == predicted_labels[i]:
                counter += 1
        return counter / float(len(actual_labels)) * 100.0

 This method is used for reading our data set from a file
 We will get a list of data samples

In [16]:
class Decidion_Tree(Decidion_Tree):      
    def loadDataSet(self, filename, data=[]):
        with open(filename, 'r') as csvfile:
            lines = csv.reader(csvfile)
            dataset = list(lines)
            for x in range(len(dataset)):
                dataset[x] = [float(i) for i in dataset[x]]
                data.append(dataset[x])

 In main function we determine the maximum tree depth, minimum nodes a tree can have
 chunk size for cross validation
 Next we build a tree, load our data set 
 split data set into 6 chunks
 Now what we do is change which data chunks we use for training and which data chunk we use for testing to test our tree 

In [18]:
if __name__ == '__main__':
    data = []
    filename = 'iris_data.csv'
    
    chunks = 6
    max_tree_lvl = 4
    min_tree_size = 10
    binary_tree = Decidion_Tree(max_tree_lvl, min_tree_size)
    binary_tree.loadDataSet(filename, data)
    chunks = binary_tree.split_into_validation_data(data, chunks)
    class_scores = list()
    for i in range(6):
        train_data = list(chunks)
        train_data.remove(chunks[i])
        train_data = sum(train_data, [])
        test_data = list()
        for row in chunks[i]:
            row_copy = list(row)
            test_data.append(row_copy)
            row_copy[-1] = None
        predicted, tree = binary_tree.decision_tree_classifier(train_data, test_data)
        actual = [row[-1] for row in chunks[i]]
        accuracy = binary_tree.prediction_accuracy(actual, predicted)
        class_scores.append(accuracy)

    print('Class Scores: %s' % class_scores)
    print('Mean Accuracy: %.3f%%' % (sum(class_scores) / float(len(class_scores))))





Class Scores: [100.0, 100.0, 100.0, 100.0, 93.75, 93.75]
Mean Accuracy: 97.917%
